In [30]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(123)

In [88]:
def init_som(x, y, n):
    """
    Initialize three-dimensional matrix x*y*n.
    :param x: map height
    :param y: map width
    :param n: number of features in input
    """
    return np.random.random((x, y, n))

init_som(2,2,2)

array([[[0.01315983, 0.02414841],
        [0.70938569, 0.92455188]],

       [[0.46733027, 0.37510915],
        [0.54286042, 0.85891684]]])

In [91]:
def get_bmu(s, som):
    """
    Get best matching unit for sample s.
    :param s: sample (size 1*number of features)
    :param som: self orginizing map (size m*n*number of features)
    :return: coordinates of best matching unit (touple (x, y))
    """
    distances = []
    for x in range(som.shape[0]):
        for y in range(som.shape[1]):
            distances += [(x, y, np.sqrt(np.sum((som[x,y,:] - s) ** 2)))]
    
    minimal = sorted(distances, key=lambda x: x[2])[0]
    return minimal[0], minimal[1]

get_bmu(np.array([5,2]), init_som(5,5,2))

(3, 4)